# Othello

Othello is a turn-based two-player strategy board game. The players take turns placing pieces, one player white and the other player black, on an 8x8 board in such a way that captures some of the opponent's pieces, with the goal of finishing the game with more pieces of their color on the board.

Every move must capture one or more of the opponent's pieces. To capture, player A places a piece adjacent to one of player B's pieces so that there is a straight line (horizontal, vertical or diagonal) of adjacent pieces that begins with one of player A's pieces, continues with one or more of player B's pieces, and ends with one of player A's pieces.

For example, if Black places a piece on square (5, 1), he will capture all of Black's pieces (5, 1) and (5, 6):